In [ ]:
# download to local
!dx login
!dx download --lightweight -fr data_structural

FileNotFoundError: [Errno 2] No such file or directory


In [8]:
import os
import pandas as pd
#get list for files have not been download successfully
#df =  pd.read_csv("file_paths.txt", header=None)
result_file_path = os.path.join('/Users/xiaoqianxiao/UKB/data/derivatives', 'output.txt')
df =  pd.read_csv("file_paths.txt")
df['filename'] = df['filepath'].str.split(r'[/]', expand=True)[5]
directory = '/Users/xiaoqianxiao/UKB/data/derivatives/downloaded_UKB'
file_names = os.listdir(directory)
df['done_files'] = df['filename'].apply(lambda x: (x in file_names)).replace((1,0),('True','False'))
df.loc[df['done_files'] == False]['filepath'].to_csv('file_paths_remain.txt', sep='\t', index=False, header=False)
df.loc[df['done_files'] == False]['filename'].to_csv(result_file_path, sep='\t', index=False, header=False)

In [10]:
import os
import zipfile
def unzip_files(source_path, unzip_path):
    # Change to the download directory
    os.chdir(download_path)
    # Iterate over all .zip files in the directory
    for f in os.listdir(download_path):
        if f.endswith('.zip'):
            # Create a directory for the unzipped contents
            destination = os.path.join(unzip_path, f[:-4])  # Remove ".zip" from filename
            os.makedirs(destination, exist_ok=True)
            
            # Unzip the file into the destination directory
            with zipfile.ZipFile(f, 'r') as zip_ref:
                zip_ref.extractall(destination)
            print(f"Extracted {f} to {destination}")
            
download_path = '/Users/xiaoqianxiao/UKB/data/derivatives/downloaded_UKB'
unzip_path = '/Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB'
unzip_files(download_path, unzip_path)

Extracted 1524972_31022_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/1524972_31022_2_0
Extracted 3155238_31022_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/3155238_31022_2_0
Extracted 4873508_31022_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/4873508_31022_2_0
Extracted 3451949_31022_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/3451949_31022_2_0
Extracted 3170803_31022_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/3170803_31022_2_0
Extracted 3954839_31022_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/3954839_31022_2_0
Extracted 5781032_31022_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/5781032_31022_2_0
Extracted 3058241_31022_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/3058241_31022_2_0
Extracted 3464428_31022_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/3464428_31022_2_0
Extracted 5252347_31022_2_0.zip to /Users/xiao

In [12]:
import os
import re
import gzip
import shutil

def rename_to_bids(base_dir, target_dir, compress=False):
    os.makedirs(target_dir, exist_ok=True)

    for subject_dir in os.listdir(base_dir):
        subject_path = os.path.join(base_dir, subject_dir)
        if not os.path.isdir(subject_path):
            continue

        # Extract subject/session IDs safely
        parts_sub = re.split(r'[_\s;]+', subject_dir)
        subject_id = parts_sub[0]
        session_id = parts_sub[2] if len(parts_sub) > 2 else "01"  # fallback if missing

        # Loop through CSV files
        for filename in os.listdir(subject_path):
            if not filename.endswith('.csv'):
                continue

            base = os.path.basename(filename)
            name, ext = os.path.splitext(base)
            parts = name.split("_")

            if len(parts) >= 3:
                metric = "".join(parts[1:-1])  # collapse underscores
                streamlines = parts[-1]

                # Build new filename
                new_filename = (
                    f"sub-{subject_id}_ses-{session_id}_"
                    f"metrics-{metric}_streamlines-{streamlines}{ext}"
                )
                if compress:
                    new_filename += ".gz"

                old_file_path = os.path.join(subject_path, filename)
                new_file_path = os.path.join(target_dir, new_filename)

                if compress:
                    with open(old_file_path, "rb") as f_in:
                        with gzip.open(new_file_path, "wb") as f_out:
                            shutil.copyfileobj(f_in, f_out)
                else:
                    shutil.copy2(old_file_path, new_file_path)

                print(f"Renamed: {old_file_path} → {new_file_path}")

# Example usage
base_dir = "/Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB"
target_dir = "/Users/xiaoqianxiao/UKB/data/derivatives/timeseries"
rename_to_bids(base_dir, target_dir, compress=False)


Renamed: /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/2542865_31022_2_0/connectome_mean_length_10M.csv → /Users/xiaoqianxiao/UKB/data/derivatives/timeseries/sub-2542865_ses-2_metrics-meanlength_streamlines-10M.csv
Renamed: /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/2542865_31022_2_0/connectome_mean_FA_10M.csv → /Users/xiaoqianxiao/UKB/data/derivatives/timeseries/sub-2542865_ses-2_metrics-meanFA_streamlines-10M.csv
Renamed: /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/2542865_31022_2_0/connectome_streamline_count_10M.csv → /Users/xiaoqianxiao/UKB/data/derivatives/timeseries/sub-2542865_ses-2_metrics-streamlinecount_streamlines-10M.csv
Renamed: /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/2542865_31022_2_0/connectome_sift2_fbc_10M.csv → /Users/xiaoqianxiao/UKB/data/derivatives/timeseries/sub-2542865_ses-2_metrics-sift2fbc_streamlines-10M.csv
Renamed: /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/5251525_31022_2_0/connectome_mean_length_10M.csv → /